# Data Factory
In this notebook, we process the data to be used in our study. We merge the World Happiness Report data together with the Our World in Data data of mental health disorders, in particular, depression and anxiety. 

In [1]:
import pandas as pd, numpy as np

In [3]:
# Read data
file_happiness = "Data Bases/WHR-Full.xls"
file_metal_health_illnesses = "Data Bases/Mental health disorders.xlsx"
dfh = pd.read_excel(file_happiness)
dfsv = pd.read_excel(file_metal_health_illnesses)

dfh.loc[dfh['Year'] == 2005, 'Generosity'] = np.nan # Only one value is redundant, so we remove it
# dfh[dfh['Year'] == 2005]['Generosity'] # Sanity check to see if the value is removed


dfsv = dfsv[dfsv['Year'] >= 2005] # Filter out years before 2005
dfsv = dfsv[dfsv['Country'] != 'China'] # In WHR data, China is splitted into China and Hong Kong, so we filter out China to avoid mismatching

# dfsv[dfsv['Country'] == 'China'] # Sanity check to see if China is filtered out   

In [4]:
# Change the names by acronyms
acronyms_h = {'Life Ladder': 'LL', 'Log GDP per capita': 'GDP', 'Social support': 'SS', 'Healthy life expectancy at birth': 'HLE', 'Freedom to make life choices': 'FMC', 'Generosity': 'G', 'Perceptions of corruption': 'PC', 'Positive affect': 'PA', 'Negative affect': 'NA'}
dfh = dfh.rename(columns=acronyms_h)


acronyms_sv = {'Depression': 'D', 'Anxiety': 'A'}
dfsv = dfsv.rename(columns=acronyms_sv)

# dfh
# dfsv

In [5]:
def countries_differences(df1, df2):
    # Extracting country names from both datasets
    countries1 = df1['Country'].unique()
    countries2 = df2['Country'].unique()

    # Converting to sets for easier comparison
    set_countries1 = set(countries1)
    set_countries2 = set(countries2)

    # Finding countries that are in one set but not the other
    countries_only_in_wrh23 = set_countries1 - set_countries2
    countries_only_in_sv = set_countries2 - set_countries1

    # Finding common countries with potential different spellings
    common_countries = set_countries1.intersection(set_countries2)
    
    return countries_only_in_wrh23, countries_only_in_sv, common_countries

countries_only_in_wrh23, countries_only_in_sv, common_countries = countries_differences(dfh, dfsv)

countries_only_in_wrh23
# countries_only_in_sv
# common_countries

{'China',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Hong Kong S.A.R. of China',
 'Ivory Coast',
 'Kosovo',
 'Somaliland region',
 'State of Palestine',
 'Taiwan Province of China',
 'Turkiye'}

In [17]:
# Define the mapping of country names from the sv dataset to the wrh23 dataset
def country_mapping(df_to_change, country_mapping):
    # Create a dictionary from the mapping for efficient replacement
    country_dict = {old_name: new_name for new_name, old_name in country_mapping}

    # Replace the country names in the 'country' column of dfsv
    df_to_change['Country'] = df_to_change['Country'].replace(country_dict)

    return df_to_change

mapping = [
    ("Congo (Brazzaville)", "Congo"),
    ("Congo (Kinshasa)", "Democratic Republic of Congo"),
    ("Ivory Coast", "Cote d'Ivoire"),
    ("State of Palestine", "Palestine"),
    ("Taiwan Province of China", "Taiwan"),
    ("Turkiye", "Turkey")
]

dfsv = country_mapping(dfsv, mapping)

# Sanity check
countries_only_in_wrh23, countries_only_in_sv, common_countries = countries_differences(dfh, dfsv)
countries_only_in_wrh23 

{'China', 'Hong Kong S.A.R. of China', 'Kosovo', 'Somaliland region'}

In [18]:
# Merge the two datasets
final_df = dfh.merge(dfsv, on=['Country', 'Year'], how='left') # Left join to keep all the data from the happiness dataset
final_df.to_excel("Data Bases/WHR-Full-Merged.xlsx", index=False)